In [1]:
# Import library and some pre-installed modules
import os
import sys
import time
import json
import requests
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import glob
import threading
import csv
import ipywidgets as widgets
from IPython.display import display, Markdown
from shapely.geometry import box, mapping
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from tqdm import tqdm
from datetime import datetime, timedelta
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from copy import deepcopy

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')
# Import the custom module

In [3]:
# Get current working directory
os.getcwd()


'c:\\Users\\Usuario\\Dev\\PhD_Thesis_Step3_OSM_Toponyms'

### Import the modules

In [4]:
# Import the custom module
from src import utils

In [5]:
# Reload the utils module to ensure any changes are reflected
import importlib
importlib.reload(utils)

<module 'src.utils' from 'c:\\Users\\Usuario\\Dev\\PhD_Thesis_Step3_OSM_Toponyms\\src\\utils.py'>

### Homogeneous Grid Cells
 - Statistical Grid (cell size of 200 x 200m) produced by Instituto Brasileiro de Geografia e Estatística (Brazilian Institute of Geography and Statistics)

  - https://geoftp.ibge.gov.br/recortes_para_fins_estatisticos/

#### Import Homogeneous Grid Cells

##### Import the grid with the aggregated data extracted from OSM via the OHSOME API

In [ ]:
# Import the statistics grid in GeoJSON format
grid = None

input_path = 'data\input_code1'

# Function for selecting and loading the GeoJSON file
def select_file(change):
    global grid
    selected_file = change['new']

    if selected_file != "Select the GeoJSON file with grid cells:":
        file_path = os.path.join(input_path, selected_file)
        try:
            with open(file_path, 'r') as file:
                grid = json.load(file)
            display("File selected with success:", selected_file)
            display("File path:", file_path)
        except FileNotFoundError:
            display("File not found:", selected_file)

# Listing available GeoJSON files
file_list = [f for f in os.listdir(input_path) if f.endswith('.geojson')]
options = ["Select the GeoJSON file with grid cells:"] + file_list

# Dropdown to select the GeoJSON file
dropdown = widgets.Dropdown(options=options)
dropdown.observe(select_file, names='value')

# Display the dropdown
display(dropdown)

In [ ]:
# Preview grid cells
grid

In [10]:
# Count the total number of grid cells in GeoJSON
total_cells = len(grid['features'])
print(f"Total grid cells in GeoJSON: {total_cells}")

Total grid cells in GeoJSON: 8652


##### Partition the original GeoJSON grid into subsets of up to 4 cells each to optimise the process

In [12]:
# Partition the original GeoJSON grid into subsets of up to 4 cells each to optimise the process

# Number of cells per batch
subset_size = 20

# Split the original grid cells into subsets
subsets = [grid['features'][i:i + subset_size] for i in range(0, len(grid['features']), subset_size)]

# Create a new FeatureCollection structure for each subset and add a batch ID ("lote_id")
grid_subsets = []
for index, subset in enumerate(subsets):
    grid_subset = {
        'type': 'FeatureCollection',
        'features': subset,
        'lote_id': f"lote{index + 1}",
        'crs': grid['crs']
    }
    grid_subsets.append(grid_subset)

In [13]:
# Calculate the total number of subsets created
total_subsets = len(grid_subsets)
print(f"Total de subsets criados: {total_subsets}")

Total de subsets criados: 433


In [ ]:
# Check the grids subsets
grid_subsets

#### Visualize the spatial distribution of the homogeneous grid cell

In [ ]:
import folium
import ipywidgets as widgets
from IPython.display import display

# Function to calculate the centroid of a polygon (original grid)
def calculate_centroid(coordinates):
    x = [p[0] for p in coordinates]
    y = [p[1] for p in coordinates]
    centroid_x = sum(x) / len(coordinates)
    centroid_y = sum(y) / len(coordinates)
    return [centroid_y, centroid_x]

# Calculate the coordinates of the centroid of the original grid
first_polygon = grid['features'][0]['geometry']['coordinates'][0][0]
centroid_coords = calculate_centroid(first_polygon)

# Function to plot a subset
def plot_subset(subset_index):
    subset_to_plot = grid_subsets[subset_index]

    # GeoJson style
    style = {'fillColor': '#8C8989', 'color': '#e31a1c', 'weight': 2}

    # Initialize the Folium map at the centroid of the original grid
    m = folium.Map(location=centroid_coords, tiles='OpenStreetMap', zoom_start=14)

    # Add GeoJson to the map
    folium.GeoJson(
        subset_to_plot,
        name=f'Grade Estatística 200m - Lote {subset_index+1}',
        tooltip=folium.GeoJsonTooltip(fields=['id', 'POP10']),
        style_function=lambda x: style
    ).add_to(m)

    # Display the map
    display(m)

# Create the drop-down list with the subset indexes
dropdown = widgets.Dropdown(
    options=[(f'Lote {i+1}', i) for i in range(len(grid_subsets))],
    description='Select a Batch:',
    disabled=False,
)

# Update the map based on the selection
widgets.interactive(plot_subset, subset_index=dropdown)

### **OHSOME API**

 - Access to features, attributes and OSM history edits using the OHSOME API (*OpenStreetMap History Data Analytics Platform*)

> - https://docs.ohsome.org/ohsome-api/v1/

In [ ]:
# Fetch metadata from the ohsome API
# This code fetches metadata from the ohsome API and handles potential JSON decoding errors.
import requests

URL = 'https://api.ohsome.org/v1/metadata'
response = requests.get(URL)

if response.status_code == 200:
    try:
        data = response.json()
        print("Dados recebidos:")
        display(data)
    except ValueError:
        print("Erro ao decodificar JSON. Conteúdo bruto:")
        display(response.text)
else:
    display(f"Erro HTTP {response.status_code}")
    print("Resposta:")
    display(response.text)

{'attribution': {'url': 'https://ohsome.org/copyrights',
  'text': '© OpenStreetMap contributors'},
 'apiVersion': '1.10.4',
 'timeout': 600.0,
 'extractRegion': {'spatialExtent': {'type': 'Polygon',
   'coordinates': [[[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]]]},
  'temporalExtent': {'fromTimestamp': '2007-10-08T00:00:00Z',
   'toTimestamp': '2025-04-06T13:00Z'},
  'replicationSequenceNumber': 110142}}

### Retrieving data from OpenStreetMap using OHSOME API and homogeneous grid cells

#### Define ET-EDGV class dictionary with respective OSM tags

In [17]:
# Novo dicionário de classes ET-EDGV com respectivas tags OSM
classe_et_edgv_to_tags = {
    'edif_ensino': [
        ('amenity', 'school'), ('amenity', 'university'),
        ('building', 'school'), ('amenity', 'kindergarten')
    ],
    'edif_saude': [
        ('amenity', 'hospital'), ('amenity', 'clinic'),
        ('building', 'hospital'), ('amenity', 'doctors'),
        ('amenity', 'dentist'), ('healthcare', '*')
    ],
    'edif_desenv_social': [
        ('amenity', 'social_facility'), ('building', 'public'),
        ('social_facility', '*')
    ],
    'edif_constr_lazer': [
        ('leisure', 'park'), ('leisure', 'sports_centre'),
        ('leisure', 'stadium'), ('amenity', 'theatre'),
        ('amenity', 'library'), ('amenity', 'community_centre'),
        ('amenity', 'arts_centre'), ('amenity', 'planetarium'),
        ('building', 'grandstand'), ('building', 'stadium'),
        ('tourism', 'museum')
    ],
    'edif_pub_civil': [
        ('building', 'public'), ('amenity', 'townhall'),
        ('office', 'government')
    ],
    'edif_turistica': [
        ('tourism', 'attraction'), ('tourism', 'artwork'),
        ('tourism', 'viewpoint'), ('amenity', 'fountain'),
        ('building', 'hotel')
    ],
    'edif_metro_ferroviaria': [
        ('railway', 'station'), ('railway', 'halt'),
        ('building', 'train_station'), ('public_transport', 'station')
    ]
}

#### Step 4 (*API Endpoint: Contributions Aggregation*): Count the total number of contributions to features with a filled-in name where a tagChange occurred:

- Count the total number of contributions to the tags of interest, aggregated by grid cell, with the attribute name filled in, considering the type of contribution (contributionType) tag change ('tagChange').

  - *contributionType available: ‘creation’, ‘deletion’, ‘tagChange’, ‘geometryChange’ ou uma combinação destes*

- Period of data retrieved: 2007-10-08 to 2025-04-06.

In [ ]:
# Step 4: tagChange com name=* — Versão final com paralelização, log CSV, CRS

# === CONFIGURAÇÕES GERAIS ===
url_tagchange = "https://api.ohsome.org/v1/contributions/count/groupBy/boundary"
params_tagchange_base = {
    'time': '2007-10-08/2025-04-06',
    'contributionType': 'tagChange'
}

output_dir = Path("data\output_code1\step4_tagchange_name")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step4.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step4.txt"

# === INICIALIZAÇÃO DO LOG ===
if not log_path.exists():
    with open(log_path, 'w', newline='') as f:
        csv.writer(f).writerow(["lote", "mensagem", "timestamp"])

def log_mensagem(lote, mensagem):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, 'a', newline='') as f:
        csv.writer(f).writerow([lote, mensagem, timestamp])

# === KEEP ALIVE ===
def keep_alive():
    while True:
        time.sleep(300)
        print("Ainda trabalhando...")
        log_mensagem("keep_alive", "Ainda trabalhando...")

threading.Thread(target=keep_alive, daemon=True).start()

# === FUNÇÃO DE CONTAGEM ===
def contar_tagchange_name(cell_geojson, tag, value):
    try:
        params = params_tagchange_base.copy()
        params.update({'bpolys': cell_geojson, 'filter': f'{tag}={value} and name=*'})
        resp = requests.post(url_tagchange, data=params)
        resp.raise_for_status()
        data = resp.json()
        return sum(r.get('value', 0) for r in data.get('groupByResult', [])[0].get('result', []))
    except Exception as e:
        return None, str(e)

# === PROCESSAMENTO DE UMA CÉLULA ===
def process_tagchange_cell(feature):
    new_feature = json.loads(json.dumps(feature))  # Deep copy
    cell_geojson = json.dumps({"type": "FeatureCollection", "features": [new_feature]})
    cell_id = new_feature['properties']['id']

    for classe, tag_list in classe_et_edgv_to_tags.items():
        total_tagchange = 0

        for tag, value in tag_list:
            for attempt in range(3):
                resultado = contar_tagchange_name(cell_geojson, tag, value)
                if resultado is not None:
                    break
                time.sleep(2 ** attempt)

            if resultado is None:
                print(f"[ERRO TAGCHANGE] Célula {cell_id} | Tag: {tag}={value}")
                log_mensagem(cell_id, f"ERRO {tag}={value}")
                continue

            total_tagchange += resultado

        new_feature['properties'][f'{classe}_name_tagchange'] = total_tagchange

    return cell_id, new_feature

# === EXECUÇÃO DOS LOTES ===
ultimo_lote = 0
if ultimo_lote_path.exists():
    with open(ultimo_lote_path, 'r') as f:
        ultimo_lote = int(f.read().strip())

for lote_index in range(ultimo_lote, len(grid_subsets)):
    start_time = time.time()
    subset = grid_subsets[lote_index]
    feature_list = subset['features']

    updated_features = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_tagchange_cell, f) for f in feature_list]
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 4)"):
            try:
                _, processed_feature = future.result()
                updated_features.append(processed_feature)
            except Exception as e:
                log_mensagem(lote_index + 1, f"FALHA: {e}")

    fc = {"type": "FeatureCollection", "features": updated_features}
    if 'crs' in grid_subsets[0]:
        fc['crs'] = grid_subsets[0]['crs']

    out_path = output_dir / f"step4_lote{lote_index + 1}.geojson"
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(fc, f)
    log_mensagem(lote_index + 1, f"SALVO {out_path.name}")

    # === CONSOLIDAÇÃO PARCIAL ===
    arquivos = sorted(glob.glob(str(output_dir / "step4_lote*.geojson")))
    todas_features = []
    for arquivo in arquivos:
        with open(arquivo, 'r', encoding='utf-8') as f:
            fc_parcial = json.load(f)
            todas_features.extend(fc_parcial['features'])

    final_fc = {"type": "FeatureCollection", "features": todas_features}
    if 'crs' in grid_subsets[0]:
        final_fc['crs'] = grid_subsets[0]['crs']

    with open(output_dir / "step4_consolidado.geojson", 'w', encoding='utf-8') as f:
        json.dump(final_fc, f)
    print("[CONSOLIDADO] step4_consolidado.geojson atualizado")
    log_mensagem(lote_index + 1, "CONSOLIDADO atualizado")

    with open(ultimo_lote_path, 'w') as f:
        f.write(str(lote_index + 1))

    tempo_msg = f"Tempo lote {lote_index + 1}: {str(timedelta(seconds=int(time.time() - start_time)))}"
    print(tempo_msg)
    log_mensagem(lote_index + 1, tempo_msg)

print("Step 4 (tagChange com name) finalizado com sucesso.")
log_mensagem("step4", "Processamento finalizado")

#### Step 5 (API Endpoint: Users Aggregation): Count the number of users (contributors) who edited features with attribute name filled in:

- Count the number of users who edited features of the OSM tags of Interest with attribute "name" attribute filled in, aggregated by grid cells.

- Period of data retrieved: 2007-10-08 to 2025-04-06.

In [ ]:
# Step 5: Users with name=* — Versão final com paralelização, logs, retry, crs

# === CONFIGURAÇÕES GERAIS ===
url_users = "https://api.ohsome.org/v1/users/count/groupBy/boundary"
params_users_base = {'time': '2007-10-08/2025-04-06'}

output_dir = Path("data\output_code1\step5_users_name")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step5.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step5.txt"

# === INICIALIZAÇÃO DO LOG ===
if not log_path.exists():
    with open(log_path, 'w', newline='') as f:
        csv.writer(f).writerow(["lote", "mensagem", "timestamp"])

def log_mensagem(lote, mensagem):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, 'a', newline='') as f:
        csv.writer(f).writerow([lote, mensagem, timestamp])

# === KEEP ALIVE ===
def keep_alive():
    while True:
        time.sleep(300)
        print("Ainda trabalhando...")
        log_mensagem("keep_alive", "Ainda trabalhando...")

threading.Thread(target=keep_alive, daemon=True).start()

# === FUNÇÃO DE CONTAGEM DE USUÁRIOS COM NAME ===
def contar_usuarios_name(cell_geojson, tag, value):
    try:
        params = params_users_base.copy()
        params.update({'bpolys': cell_geojson, 'filter': f'{tag}={value} and name=*'})
        resp = requests.post(url_users, data=params)
        resp.raise_for_status()
        data = resp.json()
        return sum(r.get('value', 0) for r in data.get('groupByResult', [])[0].get('result', []))
    except Exception as e:
        return None, str(e)

# === PROCESSAMENTO DE UMA CÉLULA ===
def process_users_cell(feature):
    new_feature = json.loads(json.dumps(feature))  # deep copy
    cell_geojson = json.dumps({"type": "FeatureCollection", "features": [new_feature]})
    cell_id = new_feature['properties']['id']

    for classe, tag_list in classe_et_edgv_to_tags.items():
        total_users_name = 0

        for tag, value in tag_list:
            for attempt in range(3):
                resultado = contar_usuarios_name(cell_geojson, tag, value)
                if resultado is not None:
                    break
                time.sleep(2 ** attempt)

            if resultado is None:
                print(f"[ERRO USERS] Célula {cell_id} | Tag: {tag}={value}")
                log_mensagem(cell_id, f"ERRO {tag}={value}")
                continue

            total_users_name += resultado

        new_feature['properties'][f'{classe}_users_name'] = total_users_name

    return cell_id, new_feature

# === EXECUÇÃO DOS LOTES ===
ultimo_lote = 0
if ultimo_lote_path.exists():
    with open(ultimo_lote_path, 'r') as f:
        ultimo_lote = int(f.read().strip())

for lote_index in range(ultimo_lote, len(grid_subsets)):
    start_time = time.time()
    subset = grid_subsets[lote_index]
    feature_list = subset['features']

    updated_features = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_users_cell, f) for f in feature_list]
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 5)"):
            try:
                _, processed_feature = future.result()
                updated_features.append(processed_feature)
            except Exception as e:
                log_mensagem(lote_index + 1, f"FALHA: {e}")

    fc = {"type": "FeatureCollection", "features": updated_features}
    if 'crs' in grid_subsets[0]:
        fc['crs'] = grid_subsets[0]['crs']

    out_path = output_dir / f"step5_lote{lote_index + 1}.geojson"
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(fc, f)
    log_mensagem(lote_index + 1, f"SALVO {out_path.name}")

    # === CONSOLIDAÇÃO PARCIAL ===
    arquivos = sorted(glob.glob(str(output_dir / "step5_lote*.geojson")))
    todas_features = []
    for arquivo in arquivos:
        with open(arquivo, 'r', encoding='utf-8') as f:
            fc_parcial = json.load(f)
            todas_features.extend(fc_parcial['features'])

    final_fc = {"type": "FeatureCollection", "features": todas_features}
    if 'crs' in grid_subsets[0]:
        final_fc['crs'] = grid_subsets[0]['crs']

    with open(output_dir / "step5_consolidado.geojson", 'w', encoding='utf-8') as f:
        json.dump(final_fc, f)
    print("[CONSOLIDADO] step5_consolidado.geojson atualizado")
    log_mensagem(lote_index + 1, "CONSOLIDADO atualizado")

    with open(ultimo_lote_path, 'w') as f:
        f.write(str(lote_index + 1))

    tempo_msg = f"Tempo lote {lote_index + 1}: {str(timedelta(seconds=int(time.time() - start_time)))}"
    print(tempo_msg)
    log_mensagem(lote_index + 1, tempo_msg)

print("Step 5 (Users with name) finalizado com sucesso.")
log_mensagem("step5", "Processamento finalizado")

#### Local tests

In [ ]:
# OHSOME API endpoint url
url_tag = "https://api.ohsome.org/v1/elements/count/groupBy/boundary/groupBy/tag"


# Start the time counter
start_time = time.time()

# Load a copy of previously created grid_subsets
grid_subset2 = grid_subsets.copy()

# Configuring basic parameters
params_base = {
    'time': '2007-10-08/2025-04-06'
}

# List to store the final results
final_results = {}

# Processar lotes
for lote_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({"type": "FeatureCollection", "features": [feature]})
        cell_id = feature['properties']['id']

        for classe, tag_list in classe_et_edgv_to_tags.items():
            total_count_classe = 0
            name_count_classe = 0
            erro_detectado = False

            for tag, value in tag_list:
                try:
                    params_total = params_base.copy()
                    params_total.update({
                        'bpolys': cell_geojson,
                        'filter': f'{tag}={value}',
                        'groupByKey': tag,
                        'groupByValues': value
                    })

                    response = requests.post(url_tag, data=params_total)
                    response.raise_for_status()
                    data = response.json()
                    count = sum(res.get('value', 0) for res in data.get('groupByResult', [])[0].get('result', []))
                    total_count_classe += count

                    # Contar apenas com nome preenchido
                    params_name = params_total.copy()
                    params_name['filter'] = f'{tag}={value} and name=*'
                    response = requests.post(url_tag, data=params_name)
                    response.raise_for_status()
                    data = response.json()
                    name_count = sum(res.get('value', 0) for res in data.get('groupByResult', [])[0].get('result', []))
                    name_count_classe += name_count

                except Exception as e:
                    erro_detectado = True
                    print(f"[ERRO] Célula {cell_id} | Tag: {tag}={value} | Erro: {e}")
                    continue

            # Armazenar resultados por classe ET-EDGV
            feature['properties'][f'{classe}_total_count'] = total_count_classe if not erro_detectado else 0
            feature['properties'][f'{classe}_name_count'] = name_count_classe if not erro_detectado else 0
            feature['properties'][f'{classe}_name_ratio'] = (
                (name_count_classe / total_count_classe) * 100 if total_count_classe > 0 else 0
            )

        final_results[cell_id] = feature['properties']

    print(f"Lote {lote_id} processado com sucesso!")

# Tempo total de execução
end_time = time.time()
total_time_seconds = end_time - start_time
print(f"Tempo total: {total_time_seconds // 60} minutos e {total_time_seconds % 60:.2f} segundos")

In [15]:
output_filename = "../outputs/OHSOME_api/02_testes_feicoes_agregadasGrade/bh/grade_id36_bh_12cells_results1_local.geojson"
# Create a new FeatureCollection to combine all the subsets
grid_subset2_results = {
    'type': 'FeatureCollection',
    'crs': grid_subset2[0]['crs'],
    'features': []
}

# Iterate over each subset and add its features to the combined FeatureCollection
for subset in grid_subset2:
    grid_subset2_results['features'].extend(subset['features'])

# Save the combined grid cells in a GeoJSON file
with open(output_filename, 'w') as file:
    json.dump(grid_subset2_results, file)